# Twitter_Text_Report
## Tiffany Marks
## October 25, 2022

In [49]:
import pandas as pd
import json
import requests
import urllib

importing things I need for this code

1. I created a Twitter Developer account and Twitter API v2 app

In [50]:
bearer_token = pd.read_csv("Twitter_Token_9-22-22", header = 0)

Put the bearer token in a file, then used pd.read_csv to read the bearer token, so the token is not opened in the code

In [51]:
ls

1018_in-class_Spotify.ipynb
EDdata_file.csv
Instagram miffanytarks_20220906/
Lecture_8-30_inclass.ipynb
Lecture_9-1_inclass.ipynb
Marks_SystemsCheck.ipynb
Oct04_in-class.ipynb
Oct18_in-class.ipynb
Sep6_in-class.ipynb
Sep8_in-class.ipynb
Sept22_in-class.ipynb
Spotify-Keys_10-18-22
Twitter_Text_Report.ipynb
Twitter_Token_9-22-22
_likes_Report.ipynb
advertisers_who_uploaded_a_contact_list_with_your_information.json
chinook.db
facebook-100084961286231/
fulldf.cvs
fulldfSaved.cvs
sqlite/


ls reveals all the files in the DataInEmat22110 Folder

In [52]:
head = {'Authorization':'Bearer {}'.format(bearer_token['Bearer_Token'].iloc[0])}

In [53]:
endpoint = 'https://api.twitter.com/2/tweets/search/recent'

head and endpoint are important as they are used for accessing the tweets

2. d) search/recent endpoint is used

In [54]:
find = '((EDTWT OR "Eating Disorder" OR "Disordered Eating") lang:en -is:retweet)'

2. a) I recently discovered a community on Twitter that bonds over their shared Eating Disorders. I am interested in looking into this topic to get a better understanding of those with Eating Disorders, what the average EDTWT user is like, their mindset, and the ethics of a community that share and bond over their mental illness. 

2. b)I first added EDTWT to my query. The community and it's users tend to have "EDTWT", code for Eating Disorder Twitter, in their usernames and tweets so they reach out to other EDTWT community members. I added the phrase Eating Disorder and Disordered Eating so I can see the general conversation around eating disorders on Twitter from those outside the community, lang:en so I can only see English Tweets and removed any retweets so I will only get original tweets. 

Just to clarify, I don't have an eating disorder myself.

With my Query, I want to search through the EDTWT community's posts and profiles. When going through this information I want to answer this question: Is allowing a community with a dangerous mental illness on a large social platform truly a negative or positive.

referenece for query search

https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query

In [55]:
queery = urllib.parse.quote(find)
expansions = 'author_id'
tweet_fields = 'id,text,author_id,created_at,public_metrics,source'
user_fields = 'id,name,username,created_at,description,verified'

2. c) These are the response fields I want to have. Id is a default for both tweet and user fields. text will show me the tweet and those tweets will show what types of discussions are happening on Edtwet, author id will give me the id of the tweet poster, created_at reveals when the tweet was published for tweet.fields, so I will know what time the tweet was made, in the user.fields created_at will reveal how long the user's been on twitter. lang gives me the tweet language, public_metrics lets me see how popular the tweet is and source tells me what device the tweet came from. descriptions will reveal the users bio, and those will help me alot with learning about the EDTWT users. Lastly, Verified lets me know if the tweet comes from a verified twitter user. These response fields include the required fields: Author ID, Author Name, Public Metrics, Creation Time

reference for different response fields

https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent

In [56]:
the_url = endpoint + '?query={}&expansions={}&tweet.fields={}&user.fields={}&max_results=100'.format(queery, expansions, tweet_fields, user_fields)

The full url needed to find the tweets that I need. I added max_results so I can get up to 100 tweets per page instead of 10

In [57]:
the_url

'https://api.twitter.com/2/tweets/search/recent?query=%28%28EDTWT%20OR%20%22Eating%20Disorder%22%20OR%20%22Disordered%20Eating%22%29%20lang%3Aen%20-is%3Aretweet%29&expansions=author_id&tweet.fields=id,text,author_id,created_at,public_metrics,source&user.fields=id,name,username,created_at,description,verified&max_results=100'

In [58]:
resp = requests.get(the_url, headers = head)

Get the url and the Bearer Token

In [59]:
resp.status_code

200

Make sure there are no errors

In [60]:
resp_dict = json.loads(resp.text)

Loads all the data in dictionary format

I won't add many full dataframes, dictionaries or list, as they take up alot of the ipynb file and aren't nice to read. I'll use the .head() for most of the dataframes

In [61]:
resp_dict['meta']

{'newest_id': '1585074649511395364',
 'oldest_id': '1585067943192592384',
 'result_count': 99,
 'next_token': 'b26v89c19zqg8o3fpzeljilo75fczuupn7yddmzil6pod'}

Meta Data: Newest tweet id, oldest tweet id, # of tweets received and the Next Token

I want more than 98 results

In [62]:
def search_repeat (query, num_pages, head):
    resp_list = []
    next_token = ''
    for i in range(0, num_pages):
        if i > 0:
            this_query = query + "&next_token={}".format(next_token)
        else:
            this_query = query

        this_resp = requests.get(this_query, headers = head)
        print(this_resp.status_code)
        this_resp_dict = json.loads(this_resp.text)
        resp_list.append(this_resp_dict)
        next_token = this_resp_dict['meta']['next_token']
    
    return resp_list

Function is instructed to grab 100 tweets based off the number of pages called

In [63]:
my_resp = search_repeat (the_url, 4, head)

200
200
200
200


2. e) Called 4 pages of 100 tweets each, grabbing only 3 will sometimes provide less than 300 tweets

In [64]:
resp_df = pd.DataFrame(my_resp)

DataFrame of all the information I requested in the query separated by keys for columns

In [65]:
pd.DataFrame(list(resp_df['meta']))

,newest_id,oldest_id,result_count,next_token
0,1585074649511395364,1585067943192592384,99,b26v89c19zqg8o3fpzeljilo75fczuupn7yddmzil6pod
1,1585067908622712832,1585062076753809408,99,b26v89c19zqg8o3fpzeljilnzie9bzu7ntn2zuc8yyd19
2,1585061932272607232,1585056258725675009,96,b26v89c19zqg8o3fpzeljildd5xprty7618343qbdgby5
3,1585056240522399744,1585050521035235330,97,b26v89c19zqg8o3fpzeljild42u72lc6r69116iiw8arh


The meta key DataFrame is organized into columns: newest_id is the most recent tweet of the page's id and oldest_id is the oldest tweet of the page's id. result_count reveals how many tweets the page collected and next_token value means there is more data and tweets for collection.

In [66]:
resp_list = list(resp_df['data'])

resp_list will hold a long list of all the information from resp_df's data key

In [67]:
resp_list_list = [pd.DataFrame(x) for x in resp_list]

resp_list_list will separate resp_list into different columns to look more organized

In [68]:
resp_data = pd.concat(resp_list_list)

In [69]:
resp_data.head()

,id,edit_history_tweet_ids,text,created_at,public_metrics,source,author_id
0,1585074649511395364,[1585074649511395364],Borzoi layout!!!\n\n☆ My name is Kat\n☆ they/t...,2022-10-26T01:03:40.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for Android,1521138170036998145
1,1585074345516277760,[1585074345516277760],@shitassfuckkk edtwt has been so toxic lately ...,2022-10-26T01:02:27.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for iPhone,1575277750482599938
2,1585074310913216512,[1585074310913216512],@shitassfuckkk edtwt is so sad to see you too ...,2022-10-26T01:02:19.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for iPhone,1314563219227373575
3,1585074201178034179,[1585074201178034179],"being both on edtwt and tcctwt, both sides hav...",2022-10-26T01:01:53.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for iPhone,1267217136377720832
4,1585074197235396609,[1585074197235396609],@shitassfuckkk edtwt is doing a good thing and...,2022-10-26T01:01:52.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter Web App,1536420223535947777


resp_data uses pd.concat to make resp_list_list into a better looking dataframe

In [70]:
public_resp = pd.DataFrame(list(resp_data['public_metrics']))

In [71]:
public_resp.head()

,retweet_count,reply_count,like_count,quote_count
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


Separated the public metrics information

In [72]:
resp_data['retweets'] = public_resp['retweet_count']
resp_data['replies'] = public_resp['reply_count']
resp_data['likes'] = public_resp['like_count']
resp_data['quotes'] = public_resp['quote_count']

Here I'm adding the public_resp columns to resp_data 

In [73]:
resp_data.head()

,id,edit_history_tweet_ids,text,created_at,public_metrics,source,author_id,retweets,replies,likes,quotes
0,1585074649511395364,[1585074649511395364],Borzoi layout!!!\n\n☆ My name is Kat\n☆ they/t...,2022-10-26T01:03:40.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for Android,1521138170036998145,0,0,0,0
1,1585074345516277760,[1585074345516277760],@shitassfuckkk edtwt has been so toxic lately ...,2022-10-26T01:02:27.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for iPhone,1575277750482599938,0,0,0,0
2,1585074310913216512,[1585074310913216512],@shitassfuckkk edtwt is so sad to see you too ...,2022-10-26T01:02:19.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for iPhone,1314563219227373575,0,0,0,0
3,1585074201178034179,[1585074201178034179],"being both on edtwt and tcctwt, both sides hav...",2022-10-26T01:01:53.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter for iPhone,1267217136377720832,0,0,0,0
4,1585074197235396609,[1585074197235396609],@shitassfuckkk edtwt is doing a good thing and...,2022-10-26T01:01:52.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Twitter Web App,1536420223535947777,0,0,0,0


In [74]:
full_data_df = resp_data.drop(columns = ['id', 'edit_history_tweet_ids', 'public_metrics'])

In [75]:
full_data_df.head()

,text,created_at,source,author_id,retweets,replies,likes,quotes
0,Borzoi layout!!!\n\n☆ My name is Kat\n☆ they/t...,2022-10-26T01:03:40.000Z,Twitter for Android,1521138170036998145,0,0,0,0
1,@shitassfuckkk edtwt has been so toxic lately ...,2022-10-26T01:02:27.000Z,Twitter for iPhone,1575277750482599938,0,0,0,0
2,@shitassfuckkk edtwt is so sad to see you too ...,2022-10-26T01:02:19.000Z,Twitter for iPhone,1314563219227373575,0,0,0,0
3,"being both on edtwt and tcctwt, both sides hav...",2022-10-26T01:01:53.000Z,Twitter for iPhone,1267217136377720832,0,0,0,0
4,@shitassfuckkk edtwt is doing a good thing and...,2022-10-26T01:01:52.000Z,Twitter Web App,1536420223535947777,0,0,0,0


I removed public metrics column, since i took all its info and separated it, and I don't need id or edit_history_tweet_ids

In [76]:
len(resp_data.index)

391

In [77]:
resp_incl_list = list(resp_df['includes'])

resp_incl_list will hold a long list of all the information in resp_df's include key

In [78]:
resp_incl_list_list = [pd.DataFrame(x) for x in resp_incl_list]

resp_incl_list_list will separate resp_incl_list into different columns to look more organized

In [79]:
resp_includes = pd.concat(resp_incl_list_list)

In [80]:
resp_includes.head()

,users
0,"{'name': 'KAT 🎃🕸 got s worded 😔✊', 'created_at..."
1,"{'name': 'REXIE!!', 'created_at': '2022-09-29T..."
2,"{'name': '𝑝𝑖𝑢 𝑝𝑖𝑢 ☆', 'created_at': '2020-10-0..."
3,"{'name': 'crakplug', 'created_at': '2020-05-31..."
4,"{'name': 'gray♡ //edtwt', 'created_at': '2022-..."


In [81]:
resp_users = pd.DataFrame(list(resp_includes['users']))

In [82]:
resp_users.head()

,name,created_at,username,id,description,verified
0,KAT 🎃🕸 got s worded 😔✊,2022-05-02T14:43:07.000Z,kiwibirdcals,1521138170036998145,"they/them || a mischievous fella, hooligan eve...",False
1,REXIE!!,2022-09-29T00:14:27.000Z,girlinurwalls,1575277750482599938,"15, she/her, edtwt, bpdtwt, shtwt, south asian",False
2,𝑝𝑖𝑢 𝑝𝑖𝑢 ☆,2020-10-09T13:47:56.000Z,Hooni_kin,1314563219227373575,sw- 154.5 || cw- ??? || gw- 120.0 || ugw- 100....,False
3,crakplug,2020-05-31T22:11:35.000Z,crakplug,1267217136377720832,ur reg 5’5 latina into theoretical physics,False
4,gray♡ //edtwt,2022-06-13T18:48:35.000Z,iamst4rv1ng,1536420223535947777,bmi: 24 -any pronouns idc- 4'10- im 13,False


After using pd.concat to organize the data into a dataframe, I had to split the information in the user column into their own columns using the list() function

In [83]:
full_data_df['user_created'] = resp_users['created_at']
full_data_df['verified'] = resp_users['verified']
full_data_df['user'] = resp_users['username']
full_data_df['name'] = resp_users['name']
full_data_df['bio'] = resp_users['description']

Adding resp_users to the full_data_df

In [84]:
full_df = pd.DataFrame(full_data_df)

In [85]:
full_df.head()

,text,created_at,source,author_id,retweets,replies,likes,quotes,user_created,verified,user,name,bio
0,Borzoi layout!!!\n\n☆ My name is Kat\n☆ they/t...,2022-10-26T01:03:40.000Z,Twitter for Android,1521138170036998145,0,0,0,0,2022-05-02T14:43:07.000Z,False,kiwibirdcals,KAT 🎃🕸 got s worded 😔✊,"they/them || a mischievous fella, hooligan eve..."
1,@shitassfuckkk edtwt has been so toxic lately ...,2022-10-26T01:02:27.000Z,Twitter for iPhone,1575277750482599938,0,0,0,0,2022-09-29T00:14:27.000Z,False,girlinurwalls,REXIE!!,"15, she/her, edtwt, bpdtwt, shtwt, south asian"
2,@shitassfuckkk edtwt is so sad to see you too ...,2022-10-26T01:02:19.000Z,Twitter for iPhone,1314563219227373575,0,0,0,0,2020-10-09T13:47:56.000Z,False,Hooni_kin,𝑝𝑖𝑢 𝑝𝑖𝑢 ☆,sw- 154.5 || cw- ??? || gw- 120.0 || ugw- 100....
3,"being both on edtwt and tcctwt, both sides hav...",2022-10-26T01:01:53.000Z,Twitter for iPhone,1267217136377720832,0,0,0,0,2020-05-31T22:11:35.000Z,False,crakplug,crakplug,ur reg 5’5 latina into theoretical physics
4,@shitassfuckkk edtwt is doing a good thing and...,2022-10-26T01:01:52.000Z,Twitter Web App,1536420223535947777,0,0,0,0,2022-06-13T18:48:35.000Z,False,iamst4rv1ng,gray♡ //edtwt,bmi: 24 -any pronouns idc- 4'10- im 13


In [86]:
len(full_df)

391

In [87]:
full_df.to_csv('fulldf.cvs', encoding='utf-8', index = False, header= True)

3. b) Exported the full_df to a csv file. Next made a copy of the csv file so I will be able to refer to this specifc full_df dataframe I collected at 7:13pm since running all the cells refreshes the dataframes with new data.

In [88]:
full_complete_df = pd.read_csv('fulldfSaved.cvs')

In [89]:
full_complete_df.head()

,author_id,created_at,source,text,retweets,replies,likes,quotes,user_created,verified,user,name,bio
0,1581508991049297920,2022-10-25T23:23:17.000Z,Twitter for iPhone,@shitassfuckkk edtwt is so cute i she is just ...,0,0,0,0,2022-10-16T04:55:10.000Z,False,mewo24517129,aubrey / shtwt edtwt,18 she/her autistic and a raging lesbian !!!NO...
1,1335241265223438337,2022-10-25T23:23:11.000Z,Twitter for iPad,edtwt watch disgusting mukbangs they make me w...,0,0,0,0,2020-12-05T15:15:07.000Z,False,fifidrms,🎞 fifi,"#143ILOVEYOU 🍂 edtwt only , idc if ur fatphobic"
2,1514172907672461316,2022-10-25T23:23:04.000Z,Twitter for iPhone,developing an eating disorder is the best thin...,0,0,0,0,2022-04-13T09:25:39.000Z,False,b1nion,binion,good mythical schmorning
3,1560220350771871744,2022-10-25T23:22:52.000Z,Twitter for iPhone,edtwt proana ricecaketwt anatwt thinspo fatspo...,0,0,0,0,2022-08-18T11:01:39.000Z,False,celimilum,celine 🐰,read carrd byf ! ♡
4,1564700974052163584,2022-10-25T23:22:30.000Z,Twitter for iPad,@shitassfuckkk edtwt is so cute I just wish sh...,0,0,0,0,2022-08-30T19:46:10.000Z,False,doe_eyed_babey,Brynne | relapse time | 5.6/65,lw 104 cw 179 ugw 110 I’m 5’5.5 dni fatphobes


This is the final dataframe! Going through each column, I believe I got the most of what I can get out of just using tweets. One limitation I have by using this format to collect tweets is the lack of images. A lot of tweets refer to images like this one for example:

In [90]:
full_complete_df['text'][10]

'edtwt did anyone play ourworld 😭😭😭 tht shit the reason i got an ed 110% look at how skinny the avatars used to be https://t.co/sgQS61vbwV'

It's not a big deal, and I can copy the url and paste it into a new tab but it does take a lot of context away from the tweet at first glance. A simple image can drastically change the intention and meaning of a tweet

Many of the users do include a lot of information in their bios: pronouns, age, political opinions, etc. But not everyone includes the same amount of information and some have irrelevant bios or bios with no information in it at all. The bio column data is very personalized compared to the other columns.

During of my process of collecting Tweets, figuring out how to save the Tweets collected by the function's first results in a csv file would have been very helpful. I've pulled over 100,000 tweets by re-running the notebook. Starting a completely new file and importing the fulldfSaved.csv file would be my next step for analyzing this data. Everything before the just helped me organize the dataframe into one I can use.